In [4]:
import json
from PIL import Image, ImageDraw
import numpy as np
import os
import math
import cv2

In [19]:
# img_folder = 'output_dir/images'
# input_folders = ['./data_20231207/Cancer', './data_20231207/Mix', './data_20231207/Warthin']
input_folders = ['./data_20240105/Cancer', './data_20240105/Mix', './data_20240105/Warthin']
# output_folders = ['output_mask_img/masks/0','output_mask_img/masks/1','output_mask_img/masks/2']
output_folders = ['new_output_mask_img/masks/0','new_output_mask_img/masks/1','new_output_mask_img/masks/2']
output_folder_img = 'new_output_mask_img/images'
# output_folder_img = 'output_mask_img/images'

test_images_folders = 'output_mask_test/images'
test_input_folders = './demo_test9'
test_output_folders = 'output_mask_test/masks'

# os.makedirs(img_folder, exist_ok=True)
for folder in output_folders:
    os.makedirs(folder, exist_ok=True)
os.makedirs(output_folder_img, exist_ok=True)

os.makedirs(test_images_folders, exist_ok=True)
os.makedirs(test_output_folders, exist_ok=True)
# for folder in test_output_folders:
#     os.makedirs(folder, exist_ok=True)

train data crop mask

In [20]:
for current_index, (input_folder, output_folder) in enumerate(zip(input_folders, output_folders)):
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            left = 0 
            right = 0 # bool if it has
            json_path = os.path.join(input_folder, filename)
            image_filename = filename.replace('.json', '.png')
            image_path = os.path.join(input_folder, image_filename)

            with open(json_path, 'r') as file:
                data = json.load(file)
            img = cv2.imread(image_path)

            image = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
            draw = ImageDraw.Draw(image)
            for shape in data['shapes']:
                label = shape['label']
                points = shape['points']
                if label == "left normal":
                    left = 1
                    left_max_x = max(point[0] for point in points)
                    left_min_x = min(point[0] for point in points)
                    left_max_y = max(point[1] for point in points)
                    left_min_y = min(point[1] for point in points)
                    left_x = left_max_x - left_min_x
                    left_y = left_max_y - left_min_y
                    continue
                elif label == "right normal":
                    right = 1
                    right_max_x = max(point[0] for point in points)
                    right_min_x = min(point[0] for point in points)
                    right_max_y = max(point[1] for point in points)
                    right_min_y = min(point[1] for point in points)
                    right_x = right_max_x - right_min_x
                    right_y = right_max_y - right_min_y
                    continue
                polygon_points = [tuple(point) for point in points]
                draw.polygon(polygon_points, fill=255)  # 在 mask 圖像上繪製多邊形

            if left == 1:
                cropped_mask_L = image.crop((math.floor(left_min_x), math.floor(left_min_y), math.ceil(left_min_x + left_x), math.ceil(left_min_y + left_y)))
                cropped_img_L = img[math.floor(left_min_y):math.ceil(left_min_y + left_y), math.floor(left_min_x):math.ceil(left_min_x + left_x)]
                L_mask_np = np.array(cropped_mask_L)
                L_img_np = np.array(cropped_img_L)
                image_filename = filename.replace('.json', '')
                output_mask_path_L = os.path.join(output_folder, image_filename + '_0.png')
                output_img_path_L = os.path.join(output_folder_img, image_filename + '_0.png')
                Image.fromarray(L_mask_np).save(output_mask_path_L)
                Image.fromarray(L_img_np).save(output_img_path_L)
                print(f"已保存 {output_mask_path_L}")
                print(f"已保存 {output_img_path_L}")
            if  right == 1:
                cropped_mask_R = image.crop((math.floor(right_min_x), math.floor(right_min_y), math.ceil(right_min_x + right_x), math.ceil(right_min_y + right_y)))
                cropped_img_R = img[math.floor(right_min_y):math.ceil(right_min_y + right_y), math.floor(right_min_x):math.ceil(right_min_x + right_x)]
                R_mask_np = np.array(cropped_mask_R)
                R_img_np = np.array(cropped_img_R)
                image_filename = filename.replace('.json', '')
                output_mask_path_R = os.path.join(output_folder, image_filename + '_1.png')
                output_img_path_R = os.path.join(output_folder_img, image_filename + '_1.png')
                Image.fromarray(R_mask_np).save(output_mask_path_R)
                Image.fromarray(R_img_np).save(output_img_path_R)
                print(f"已保存 {output_mask_path_R}")
                print(f"已保存 {output_img_path_R}")


已保存 new_output_mask_img/masks/0\b86-12020577_2_0.png
已保存 new_output_mask_img/images\b86-12020577_2_0.png
已保存 new_output_mask_img/masks/0\b86-12020577_2_1.png
已保存 new_output_mask_img/images\b86-12020577_2_1.png
已保存 new_output_mask_img/masks/0\b86-12020577_3_1.png
已保存 new_output_mask_img/images\b86-12020577_3_1.png
已保存 new_output_mask_img/masks/0\b86-12020577_4_0.png
已保存 new_output_mask_img/images\b86-12020577_4_0.png
已保存 new_output_mask_img/masks/0\b86-12020577_4_1.png
已保存 new_output_mask_img/images\b86-12020577_4_1.png
已保存 new_output_mask_img/masks/0\b86-12020577_5_0.png
已保存 new_output_mask_img/images\b86-12020577_5_0.png
已保存 new_output_mask_img/masks/0\b86-12020577_5_1.png
已保存 new_output_mask_img/images\b86-12020577_5_1.png
已保存 new_output_mask_img/masks/0\b86-12020577_6_0.png
已保存 new_output_mask_img/images\b86-12020577_6_0.png
已保存 new_output_mask_img/masks/0\b86-12020577_6_1.png
已保存 new_output_mask_img/images\b86-12020577_6_1.png
已保存 new_output_mask_img/masks/0\b86-12020577_7_0.png
已保

test data crop mask

In [14]:
for filename in os.listdir(test_input_folders):
    if filename.endswith('.json'):
        left = 0 
        right = 0 # bool if it has
        json_path = os.path.join(test_input_folders, filename)
        image_filename = filename.replace('.json', '.png')
        image_path = os.path.join(test_input_folders, image_filename)

        with open(json_path, 'r') as file:
            data = json.load(file)
        img = cv2.imread(image_path)

        image = Image.new('L', (data['imageWidth'], data['imageHeight']), 0)
        draw = ImageDraw.Draw(image)
        for shape in data['shapes']:
            label = shape['label']
            points = shape['points']
            if label == "left normal":
                left = 1
                left_max_x = max(point[0] for point in points)
                left_min_x = min(point[0] for point in points)
                left_max_y = max(point[1] for point in points)
                left_min_y = min(point[1] for point in points)
                left_x = left_max_x - left_min_x
                left_y = left_max_y - left_min_y
                continue
            elif label == "right normal":
                right = 1
                right_max_x = max(point[0] for point in points)
                right_min_x = min(point[0] for point in points)
                right_max_y = max(point[1] for point in points)
                right_min_y = min(point[1] for point in points)
                right_x = right_max_x - right_min_x
                right_y = right_max_y - right_min_y
                continue
            polygon_points = [tuple(point) for point in points]
            draw.polygon(polygon_points, fill=255)  # 在 mask 圖像上繪製多邊形

        if left == 1:
            cropped_mask_L = image.crop((math.floor(left_min_x), math.floor(left_min_y), math.ceil(left_min_x + left_x), math.ceil(left_min_y + left_y)))
            cropped_img_L = img[math.floor(left_min_y):math.ceil(left_min_y + left_y), math.floor(left_min_x):math.ceil(left_min_x + left_x)]
            L_mask_np = np.array(cropped_mask_L)
            L_img_np = np.array(cropped_img_L)
            image_filename = filename.replace('.json', '')
            output_mask_path_L = os.path.join(test_output_folders, image_filename + '_0.png')
            output_img_path_L = os.path.join(test_images_folders, image_filename + '_0.png')
            Image.fromarray(L_mask_np).save(output_mask_path_L)
            Image.fromarray(L_img_np).save(output_img_path_L)
            print(f"已保存 {output_mask_path_L}")
            print(f"已保存 {output_img_path_L}")
        if  right == 1:
            cropped_mask_R = image.crop((math.floor(right_min_x), math.floor(right_min_y), math.ceil(right_min_x + right_x), math.ceil(right_min_y + right_y)))
            cropped_img_R = img[math.floor(right_min_y):math.ceil(right_min_y + right_y), math.floor(right_min_x):math.ceil(right_min_x + right_x)]
            R_mask_np = np.array(cropped_mask_R)
            R_img_np = np.array(cropped_img_R)
            image_filename = filename.replace('.json', '')
            output_mask_path_R = os.path.join(test_output_folders, image_filename + '_1.png')
            output_img_path_R = os.path.join(test_images_folders, image_filename + '_1.png')
            Image.fromarray(R_mask_np).save(output_mask_path_R)
            Image.fromarray(R_img_np).save(output_img_path_R)
            print(f"已保存 {output_mask_path_R}")
            print(f"已保存 {output_img_path_R}")

已保存 output_mask_test/masks\b58-15991078_11_0.png
已保存 output_mask_test/images\b58-15991078_11_0.png
已保存 output_mask_test/masks\b58-15991078_11_1.png
已保存 output_mask_test/images\b58-15991078_11_1.png
已保存 output_mask_test/masks\b59-15762303_9_0.png
已保存 output_mask_test/images\b59-15762303_9_0.png
已保存 output_mask_test/masks\b59-15762303_9_1.png
已保存 output_mask_test/images\b59-15762303_9_1.png
已保存 output_mask_test/masks\b60-02986412_8_0.png
已保存 output_mask_test/images\b60-02986412_8_0.png
已保存 output_mask_test/masks\b60-02986412_8_1.png
已保存 output_mask_test/images\b60-02986412_8_1.png
已保存 output_mask_test/masks\b61-08688114_6_0.png
已保存 output_mask_test/images\b61-08688114_6_0.png
已保存 output_mask_test/masks\b61-08688114_6_1.png
已保存 output_mask_test/images\b61-08688114_6_1.png
已保存 output_mask_test/masks\b70-09071962_9_0.png
已保存 output_mask_test/images\b70-09071962_9_0.png
已保存 output_mask_test/masks\b70-09071962_9_1.png
已保存 output_mask_test/images\b70-09071962_9_1.png
已保存 output_mask_test/masks